之所以叫Adavanced只是不想和前面的Regression重名



# SVR

导入2001年至2010年的输入变量，距平并作标准化处理

In [1]:
import numpy as np

origin = np.load('total_used.npy')[:3287]

loss = np.isnan(origin)

origin[loss] = 0

year_start = [0]

for year in range(2002, 2010):

    if (year - 1) % 4 == 0:

        year_start += [year_start[-1] + 366]

    else:

        year_start += [year_start[-1] + 365]

year_start = np.array(year_start)

year_leap = [1460, 2921]

day_average = np.zeros((366, 4763))

for i in range(365):

    day_average[i] = np.sum(origin[year_start + i], axis = 0) / np.sum(1 - loss[year_start + i], axis = 0)  # 求平均时不考虑缺失值

    origin[year_start + i] -= day_average[i]  # 距平

day_average[365] = np.sum(origin[year_leap], axis = 0) / np.sum(1 - loss[year_leap], axis = 0)  # 对于闰年中一年的第366天求平均

origin[year_leap] -= day_average[365]  # 距平

origin[loss] = 0  # 对于缺失值认为等于历史平均值，历史平均值的距平为0

origin_std = np.std(origin, axis = 0) / ((np.sum(1 - loss, axis = 0) / 3287) ** 0.5)  # 距平相当于已经做了减均值处理，
                                                                                      # 此处只需要除以方差即可

origin /= origin_std

将输入变量降至1000维

In [2]:
from sklearn.decomposition import PCA

pca = PCA(n_components=1000)

pca.fit(origin)

X = pca.fit_transform(origin)

X = X[:-1]

导入输出变量，距平，并将其相对输入变量后移一位

In [3]:
y = np.load('y.npy')[:3287, 0]

loss_y = np.isnan(y)

y[loss_y] = 0

y_day_average = np.zeros(366)

for i in range(365):

    y_day_average[i] = np.sum(y[year_start + i]) / np.sum(1 - loss_y[year_start + i])

    y[year_start + i] -= y_day_average[i]

y_day_average[365] = np.sum(y[year_leap]) / np.sum(1 - loss_y[year_leap])

y[year_leap] -= y_day_average[365]

In [4]:
y = y[1:]

loss_y = loss_y[1:]

只拟合输出变量中观测值不缺失的样本

In [5]:
use_y = loss_y == 0

new_y = y[use_y]

new_X = X[use_y]

采用5折交叉验证发验证（因为5折要比10折省一半的时间……）

In [ ]:
from sklearn.svm import SVR
import sklearn.model_selection as sk_model_selection

for i in (0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1):
    for j in (0.0001, 0.001, 0.01, 0.1, 1, 10):
        clf = SVR(kernel='linear', C=i, epsilon=j)
        print(i, j, abs(np.average(sk_model_selection.cross_val_score(clf, new_X, new_y, scoring='neg_mean_absolute_error', cv = 5))))

1e-06 0.0001 3.7762624396219513
1e-06 0.001 3.77624302933814
1e-06 0.01 3.776365271062583
1e-06 0.1 3.7767908913985075
1e-06 1 3.7844151617155255
1e-06 10 4.074643100257235
1e-05 0.0001 2.9021552059444287
1e-05 0.001 2.9021721098516577
1e-05 0.01 2.902243458340473
1e-05 0.1 2.9018608006836013
1e-05 1 2.9301059007682118
1e-05 10 3.7946043069161006
0.0001 0.0001 2.4413764363991306
0.0001 0.001 2.4413380040254546
0.0001 0.01 2.440972829656327
0.0001 0.1 2.4391477116587845
0.0001 1 2.4409796299006805
0.0001 10 3.2852675050901703
0.001 0.0001 2.3644194409334114
0.001 0.001 2.3643485657191423
0.001 0.01 2.363618572725393
0.001 0.1 2.356978295625651
0.001 1 2.302852903404971
0.001 10 3.003806169591558
0.01 0.0001 2.6510182947028733
0.01 0.001 2.6509491004085666
0.01 0.01 2.6502939488836637
0.01 0.1 2.6406369922885995
0.01 1 2.5417877634544817
0.01 10 2.966498802805132
0.1 0.0001 2.874843771622226
0.1 0.001 2.8745990834029507
0.1 0.01 2.8727003923920797
0.1 0.1 2.8569866334155547
0.1 1 2.72265

采用线性回归和SVR做对比

In [14]:
from sklearn import linear_model

clf = linear_model.LinearRegression()

print(abs(np.average(sk_model_selection.cross_val_score(clf, new_X, new_y, scoring='neg_mean_absolute_error', cv = 5))))

2.7311729783382788


## valid

导入2001年至2009年的输入变量，距平并作标准化处理

In [57]:
import numpy as np

origin = np.load('total_used.npy')[:3287]

loss = np.isnan(origin)

origin[loss] = 0

year_start = [0]

for year in range(2002, 2010):

    if (year - 1) % 4 == 0:

        year_start += [year_start[-1] + 366]

    else:

        year_start += [year_start[-1] + 365]

year_start = np.array(year_start)

year_leap = [1460, 2921]

day_average = np.zeros((366, 4763))

for i in range(365):

    day_average[i] = np.sum(origin[year_start + i], axis = 0) / np.sum(1 - loss[year_start + i], axis = 0)  # 求平均时不考虑缺失值

    origin[year_start + i] -= day_average[i]  # 距平

day_average[365] = np.sum(origin[year_leap], axis = 0) / np.sum(1 - loss[year_leap], axis = 0)  # 对于闰年中一年的第366天求平均

origin[year_leap] -= day_average[365]  # 距平

origin[loss] = 0  # 对于缺失值认为等于历史平均值，历史平均值的距平为0

origin_std = np.std(origin, axis = 0) / ((np.sum(1 - loss, axis = 0) / 3287) ** 0.5)  # 距平相当于已经做了减均值处理，
                                                                                      # 此处只需要除以方差即可

origin /= origin_std

将输入变量降至1000维

In [58]:
from sklearn.decomposition import PCA

pca = PCA(n_components=1000)

pca.fit(origin)

X = pca.fit_transform(origin)

X = X[:-1]

导入输出变量，距平，并将其相对输入变量后移一位

In [59]:
y = np.load('y.npy')[:3287, 0]

loss_y = np.isnan(y)

y[loss_y] = 0

y_day_average = np.zeros(366)

for i in range(365):

    y_day_average[i] = np.sum(y[year_start + i]) / np.sum(1 - loss_y[year_start + i])

    y[year_start + i] -= y_day_average[i]

y_day_average[365] = np.sum(y[year_leap]) / np.sum(1 - loss_y[year_leap])

y[year_leap] -= y_day_average[365]

In [60]:
y = y[1:]

loss_y = loss_y[1:]

只拟合输出变量中观测值不缺失的样本

In [61]:
use_y = loss_y == 0

new_y = y[use_y]

new_X = X[use_y]

In [7]:
from sklearn.svm import SVR

clf = SVR(kernel='linear', C=0.001, epsilon=1)

clf.fit(new_X, new_y)

SVR(C=0.001, cache_size=200, coef0=0.0, degree=3, epsilon=1,
  gamma='auto_deprecated', kernel='linear', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [43]:
from sklearn import linear_model

reg = linear_model.LinearRegression()

reg.fit(new_X, new_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

导入2009年12月31日（含）至2010年12月30日的输入变量（含），使用训练集的历史平均值和训练集的pca基底对其进行预处理

In [10]:
origin_valid = np.load('total_used.npy')[3286:3651]

loss_valid = np.isnan(origin_valid)

for i in range(364):

    origin_valid[i + 1] -= day_average[i]  # origin_valid中第2天才是一年中的第1天

origin_valid[0] -= day_average[-2]

origin_valid[loss_valid] = 0  # 将缺失值设为历史平均值

origin_valid /= origin_std

X_valid = origin_valid @ pca.components_.T

预测并加回历史平均

In [13]:
y_valid = clf.predict(X_valid) + y_day_average[:-1]  # 2010年不是闰年

导入2010年1月1日至2010年12月31日的输出变量，并做验证

In [46]:
y_true = np.load('y.npy')[3287:3652, 0]

mae = np.average(abs(y_valid - y_true))
mae

2.8303017062124693

In [47]:
mae = np.average(abs(y_valid - y_day_average[:-1]))
mae

4.393351753572736

In [44]:
y_linearreg = reg.predict(X_valid) + y_day_average[:-1]

In [48]:
mae = np.average(abs(y_valid - y_linearreg))
mae

1.3520955584137

In [45]:
np.savetxt('y.txt', np.vstack((y_true, y_valid, y_day_average[:-1], y_linearreg)).T)

# RBF SVR

In [56]:
from sklearn.svm import SVR
import sklearn.model_selection as sk_model_selection

for i in (0.01, 0.1, 1, 10, 100, 1000):
    for j in (0.0001, 0.001, 0.01, 0.1, 1, 10, 100):
        clf = SVR(gamma='scale', C=i, epsilon=j)
        print(i, j, abs(np.average(sk_model_selection.cross_val_score(clf, new_X, new_y, scoring='neg_mean_absolute_error', cv = 5))))

0.01 0.0001 3.927452941164313
0.01 0.001 3.927427268323316
0.01 0.01 3.927349540945368
0.01 0.1 3.928490327463261
0.01 1 3.9308950558193487
0.01 10 4.109513596855781
0.01 100 4.163456972059292
0.1 0.0001 3.212793751753722
0.1 0.001 3.212775097777663
0.1 0.01 3.2126803331749563
0.1 0.1 3.2140051394332545
0.1 1 3.236185187659436
0.1 10 3.953866881460132
0.1 100 4.163456972059292
1 0.0001 2.634410043439544
1 0.001 2.634410598022209
1 0.01 2.634460322730633
1 0.1 2.6341091336239004
1 1 2.638778782882066
1 10 3.5379579064799835
1 100 4.163456972059292
10 0.0001 2.470599627249307
10 0.001 2.4705877440944457
10 0.01 2.4704934205193463
10 0.1 2.469470649384615
10 1 2.471937338769594
10 10 3.2858677866909964
10 100 4.163456972059292
100 0.0001 2.500093156993598
100 0.001 2.500063217240568
100 0.01 2.4997946994299705
100 0.1 2.4974561652699885
100 1 2.4890571206253638
100 10 3.2858677866909964
100 100 4.163456972059292
1000 0.0001 2.500093156993598
1000 0.001 2.500063217240568
1000 0.01 2.499794

In [62]:
from sklearn.svm import SVR

clf = SVR(gamma='scale', C=10, epsilon=0.01)

clf.fit(new_X, new_y)

SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.01, gamma='scale',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [65]:
y_valid = clf.predict(X_valid) + y_day_average[:-1]
np.savetxt('RBFSVM.txt', y_valid)

In [66]:
y_true = np.load('y.npy')[3287:3652, 0]

mae = np.average(abs(y_valid - y_true))
mae

3.096349622588674

# Decison Tree

In [9]:
from sklearn.tree import DecisionTreeRegressor
import sklearn.model_selection as sk_model_selection

for i in range(2, 9):
    clf = DecisionTreeRegressor(max_depth=i)
    print(i, abs(np.average(sk_model_selection.cross_val_score(clf, new_X, new_y, scoring='neg_mean_absolute_error', cv = 5))))

2 3.4894812269091657
3 3.391846277124871
4 3.4021654441879305
5 3.4473692377516643
6 3.6057647638867047
7 3.775876609274878
8 3.9164711889007413


In [12]:
from sklearn.tree import DecisionTreeRegressor

clf = DecisionTreeRegressor(max_depth=3)

clf.fit(new_X, new_y)

DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [14]:
y_valid = clf.predict(X_valid) + y_day_average[:-1]

np.savetxt('DT.txt', y_valid)

In [15]:
y_true = np.load('y.npy')[3287:3652, 0]

mae = np.average(abs(y_valid - y_true))
mae

3.6148419242727843

# Random Forest

In [18]:
from sklearn.ensemble import RandomForestRegressor
import sklearn.model_selection as sk_model_selection

for i in range(2, 17):
    clf = RandomForestRegressor(max_depth=i, n_estimators=100)
    print(i, abs(np.average(sk_model_selection.cross_val_score(clf, new_X, new_y, scoring='neg_mean_absolute_error', cv = 5))))

2 3.4319333325581356
3 3.259596689628288
4 3.157117372954301
5 3.1239349683423265
6 3.1111357027615574
7 3.091303560417791
8 3.0860126801880483
9 3.0960295797533877
10 3.091931452303018
11 3.0965109319634587
12 3.0988412054890375
13 3.092423547554343
14 3.1082729912058396
15 3.075739671587096
16 3.1034251508104287


In [19]:
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor(max_depth=8, n_estimators=100)

clf.fit(new_X, new_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [20]:
y_valid = clf.predict(X_valid) + y_day_average[:-1]

np.savetxt('RF.txt', y_valid)

In [21]:
y_true = np.load('y.npy')[3287:3652, 0]

mae = np.average(abs(y_valid - y_true))
mae

2.086623246458305

# Neural Network

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1000, 3)
        self.fc2 = nn.Linear(3, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [23]:
X_train = torch.from_numpy(new_X).float()
X_test = torch.from_numpy(X_valid).float()
y_train = torch.from_numpy(np.array([new_y]).T).float()

In [39]:
net = Net()

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.00001)

In [40]:
for epoch in range(100000):
    outputs = net(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 500 == 0:
        print(epoch + 1, float(loss[0]))
        
print('Finished Training')

d:\python36\lib\site-packages\ipykernel_launcher.py:8: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  


1 27.78514289855957
501 26.5269775390625
1001 25.36873435974121
1501 24.271699905395508
2001 23.222030639648438
2501 22.221590042114258
3001 21.2681827545166
3501 20.3751163482666
4001 19.526918411254883
4501 18.72639274597168
5001 17.971839904785156
5501 17.263669967651367
6001 16.597654342651367
6501 15.964357376098633
7001 15.366880416870117
7501 14.801788330078125
8001 14.26591682434082
8501 13.756755828857422
9001 13.268279075622559
9501 12.799700736999512
10001 12.351975440979004
10501 11.921003341674805
11001 11.50158977508545
11501 11.098526954650879
12001 10.706625938415527
12501 10.328885078430176
13001 9.964238166809082
13501 9.60960865020752
14001 9.26612377166748
14501 8.9329195022583
15001 8.609207153320312
15501 8.29646110534668
16001 7.99355936050415
16501 7.7023115158081055
17001 7.421689987182617
17501 7.152163028717041
18001 6.895515441894531
18501 6.64851713180542
19001 6.408320903778076
19501 6.180277347564697
20001 5.961472034454346
20501 5.7553229331970215
21001 

In [51]:
y_valid = net(X_test).detach().numpy().T[0] + y_day_average[:-1]

np.savetxt('NN.txt', y_valid)

In [52]:
y_true = np.load('y.npy')[3287:3652, 0]

mae = np.average(abs(y_valid - y_true))
mae

0.7146772026689084